In [1]:
from openai import OpenAI
import json
import uuid

In [ ]:
client = OpenAI(api_key="sk-fff", base_url="https://api.deepseek.com")

In [3]:
systemPrompt = """You are ZenCode, an expert AI assistant specializing in generating React applications that strictly adhere to enterprise design standards and component libraries.

<repository_context>
  You will receive a serialized context of the current repository's generated code state. This includes:
  - File structure and contents of previously generated code
  - Current state of all files and directories
  - Previous modifications and their history
  
  CRITICAL:
  - All file operations must be performed relative to this existing context
  - Maintain consistency with previously generated code
  - Consider dependencies and relationships between existing files
  - Ensure backwards compatibility when modifying existing files
</repository_context>

<system_constraints>
  You are operating in WebContainer, an in-browser Node.js runtime that emulates a Linux system. Key limitations:
  - Runs in browser, not a full Linux system
  - Can only execute browser-compatible code (JS, WebAssembly)
  - Python limited to standard library only (NO pip)
  - No native binary execution or C/C++ compilation
  - Git not available
  - For databases, use browser-compatible options (libsql, sqlite)
</system_constraints>

<enterprise_context>
  You will receive:
  1. Top-K relevant internal components from our vector database (Pinecone)
  2. Global CSS standards and design system guidelines
  3. Approved list of npm packages
  
  CRITICAL REQUIREMENTS:
  - ONLY use provided internal components from the enterprise library
  - Strictly follow enterprise design standards
  - Only use approved npm packages. Do not use any external npm packages not used in the given context. 
  - Focus on rapid prototyping by reusing existing components
  - Maintain consistent styling and UX patterns
</enterprise_context>

<response_format>
  Generate responses in JSON format following the StepType enum:
  - CreateFile (0): New files
  - CreateFolder (1): New directories
  - EditFile (2): Modify files
  - DeleteFile (3): Remove files

  Each step must include:
  - id: Unique integer
  - title: Step description
  - description: Detailed explanation
  - type: StepType enum value
  - content: File content or command
  - path: Target file/folder path
</response_format>

<code_formatting>
  - Use 2 spaces for indentation
  - Follow enterprise code style guide
  - Split functionality into modular components
  - Keep files small and focused
  - Use proper TypeScript types
</code_formatting>

IMPORTANT:
1. Think holistically before generating responses
2. Consider all file dependencies and impacts
3. Always use latest file modifications
4. Install dependencies first
5. Never re-run dev server if already running
6. Provide complete, untruncated code
7. Focus on reusing enterprise components
   
"""

In [4]:
codeBasePrompt = """
Project Files:\n\nThe following is a list of all project files and their complete contents that are currently visible and accessible to you.\n\neslint.config.js:\n```\nimport js from '@eslint/js';\nimport globals from 'globals';\nimport reactHooks from 'eslint-plugin-react-hooks';\nimport reactRefresh from 'eslint-plugin-react-refresh';\nimport tseslint from 'typescript-eslint';\n\nexport default tseslint.config(\n  { ignores: ['dist'] },\n  {\n    extends: [js.configs.recommended, ...tseslint.configs.recommended],\n    files: ['**/*.{ts,tsx}'],\n    languageOptions: {\n      ecmaVersion: 2020,\n      globals: globals.browser,\n    },\n    plugins: {\n      'react-hooks': reactHooks,\n      'react-refresh': reactRefresh,\n    },\n    rules: {\n      ...reactHooks.configs.recommended.rules,\n      'react-refresh/only-export-components': [\n        'warn',\n        { allowConstantExport: true },\n      ],\n    },\n  }\n);\n\n```\n\nindex.html:\n```\n<!doctype html>\n<html lang=\"en\">\n  <head>\n    <meta charset=\"UTF-8\" />\n    <link rel=\"icon\" type=\"image/svg+xml\" href=\"/vite.svg\" />\n    <meta name=\"viewport\" content=\"width=device-width, initial-scale=1.0\" />\n    <title>calculator</title>\n  </head>\n  <body>\n    <div id=\"root\"></div>\n    <script type=\"module\" src=\"/src/main.tsx\"></script>\n  </body>\n</html>\n\n```\n\npackage.json:\n```\n{\n  \"name\": \"vite-react-typescript-starter\",\n  \"private\": true,\n  \"version\": \"0.0.0\",\n  \"type\": \"module\",\n  \"scripts\": {\n    \"dev\": \"vite\",\n    \"build\": \"vite build\",\n    \"lint\": \"eslint .\",\n    \"preview\": \"vite preview\"\n  },\n  \"dependencies\": {\n    \"lucide-react\": \"^0.344.0\",\n    \"react\": \"^18.3.1\",\n    \"react-dom\": \"^18.3.1\"\n  },\n  \"devDependencies\": {\n    \"@eslint/js\": \"^9.9.1\",\n    \"@types/react\": \"^18.3.5\",\n    \"@types/react-dom\": \"^18.3.0\",\n    \"@vitejs/plugin-react\": \"^4.3.1\",\n    \"autoprefixer\": \"^10.4.18\",\n    \"eslint\": \"^9.9.1\",\n    \"eslint-plugin-react-hooks\": \"^5.1.0-rc.0\",\n    \"eslint-plugin-react-refresh\": \"^0.4.11\",\n    \"globals\": \"^15.9.0\",\n    \"postcss\": \"^8.4.35\",\n    \"tailwindcss\": \"^3.4.1\",\n    \"typescript\": \"^5.5.3\",\n    \"typescript-eslint\": \"^8.3.0\",\n    \"vite\": \"^5.4.2\"\n  }\n}\n\n```\n\npostcss.config.js:\n```\nexport default {\n  plugins: {\n    tailwindcss: {},\n    autoprefixer: {},\n  },\n};\n\n```\n\nsrc/App.tsx:\n```\nimport React, { useState } from 'react';\nimport { Equal, Delete, Plus, Minus, X, Divide, Moon, Sun } from 'lucide-react';\n\nfunction App() {\n  const [display, setDisplay] = useState('0');\n  const [equation, setEquation] = useState('');\n  const [shouldResetDisplay, setShouldResetDisplay] = useState(false);\n  const [isDark, setIsDark] = useState(false);\n\n  const handleNumber = (num: string) => {\n    if (shouldResetDisplay) {\n      setDisplay(num);\n      setShouldResetDisplay(false);\n    } else {\n      setDisplay(display === '0' ? num : display + num);\n    }\n  };\n\n  const handleOperator = (operator: string) => {\n    setShouldResetDisplay(true);\n    setEquation(display + ' ' + operator + ' ');\n  };\n\n  const handleEqual = () => {\n    try {\n      const result = eval(equation + display);\n      setDisplay(String(result));\n      setEquation('');\n    } catch (error) {\n      setDisplay('Error');\n    }\n    setShouldResetDisplay(true);\n  };\n\n  const handleClear = () => {\n    setDisplay('0');\n    setEquation('');\n    setShouldResetDisplay(false);\n  };\n\n  const toggleTheme = () => {\n    setIsDark(!isDark);\n    document.documentElement.classList.toggle('dark');\n  };\n\n  const CalcButton = ({ onClick, children, className = '' }: { \n    onClick: () => void;\n    children: React.ReactNode;\n    className?: string;\n  }) => (\n    <button\n      onClick={onClick}\n      className={`p-4 text-lg font-semibold rounded-xl transition-all duration-200 \n      hover:bg-opacity-90 active:scale-95 ${className}`}\n    >\n      {children}\n    </button>\n  );\n\n  return (\n    <div className={`min-h-screen flex items-center justify-center p-4\n      ${isDark ? 'bg-gradient-to-br from-gray-900 to-gray-800' : 'bg-gradient-to-br from-blue-100 to-purple-100'}`}>\n      <div className={`rounded-2xl shadow-2xl w-full max-w-xs p-6\n        ${isDark ? 'bg-gray-800' : 'bg-white'}`}>\n        <div className=\"flex justify-between items-center mb-4\">\n          <button\n            onClick={toggleTheme}\n            className={`p-2 rounded-lg transition-colors\n              ${isDark ? 'text-gray-300 hover:bg-gray-700' : 'text-gray-600 hover:bg-gray-100'}`}\n          >\n            {isDark ? <Sun size={20} /> : <Moon size={20} />}\n          </button>\n          <div className=\"text-right flex-1\">\n            <div className={`h-6 text-sm ${isDark ? 'text-gray-400' : 'text-gray-500'}`}>\n              {equation}\n            </div>\n            <div className={`text-3xl font-bold h-12 overflow-hidden\n              ${isDark ? 'text-white' : 'text-gray-900'}`}>\n              {display}\n            </div>\n          </div>\n        </div>\n        \n        <div className=\"grid grid-cols-4 gap-2\">\n          <CalcButton\n            onClick={handleClear}\n            className=\"bg-red-500 text-white col-span-2 dark:bg-red-600\"\n          >\n            <div className=\"flex items-center justify-center gap-1\">\n              <Delete size={20} /> Clear\n            </div>\n          </CalcButton>\n          <CalcButton\n            onClick={() => handleOperator('/')}\n            className={isDark ? 'bg-gray-700 text-gray-200' : 'bg-gray-200'}\n          >\n            <Divide size={20} />\n          </CalcButton>\n          <CalcButton\n            onClick={() => handleOperator('*')}\n            className={isDark ? 'bg-gray-700 text-gray-200' : 'bg-gray-200'}\n          >\n            <X size={20} />\n          </CalcButton>\n\n          {[7, 8, 9].map((num) => (\n            <CalcButton\n              key={num}\n              onClick={() => handleNumber(String(num))}\n              className={isDark ? 'bg-gray-700 text-gray-200' : 'bg-gray-100'}\n            >\n              {num}\n            </CalcButton>\n          ))}\n          <CalcButton\n            onClick={() => handleOperator('-')}\n            className={isDark ? 'bg-gray-700 text-gray-200' : 'bg-gray-200'}\n          >\n            <Minus size={20} />\n          </CalcButton>\n\n          {[4, 5, 6].map((num) => (\n            <CalcButton\n              key={num}\n              onClick={() => handleNumber(String(num))}\n              className={isDark ? 'bg-gray-700 text-gray-200' : 'bg-gray-100'}\n            >\n              {num}\n            </CalcButton>\n          ))}\n          <CalcButton\n            onClick={() => handleOperator('+')}\n            className={isDark ? 'bg-gray-700 text-gray-200' : 'bg-gray-200'}\n          >\n            <Plus size={20} />\n          </CalcButton>\n\n          {[1, 2, 3].map((num) => (\n            <CalcButton\n              key={num}\n              onClick={() => handleNumber(String(num))}\n              className={isDark ? 'bg-gray-700 text-gray-200' : 'bg-gray-100'}\n            >\n              {num}\n            </CalcButton>\n          ))}\n          <CalcButton\n            onClick={handleEqual}\n            className=\"bg-blue-500 text-white row-span-2 dark:bg-blue-600\"\n          >\n            <Equal size={20} />\n          </CalcButton>\n\n          <CalcButton\n            onClick={() => handleNumber('0')}\n            className={`col-span-2 ${isDark ? 'bg-gray-700 text-gray-200' : 'bg-gray-100'}`}\n          >\n            0\n          </CalcButton>\n          <CalcButton\n            onClick={() => handleNumber('.')}\n            className={isDark ? 'bg-gray-700 text-gray-200' : 'bg-gray-100'}\n          >\n            .\n          </CalcButton>\n        </div>\n      </div>\n    </div>\n  );\n}\n\nexport default App;\n```\n\nsrc/index.css:\n```\n@tailwind base;\n@tailwind components;\n@tailwind utilities;\n\n```\n\nsrc/main.tsx:\n```\nimport { StrictMode } from 'react';\nimport { createRoot } from 'react-dom/client';\nimport App from './App.tsx';\nimport './index.css';\n\ncreateRoot(document.getElementById('root')!).render(\n  <StrictMode>\n    <App />\n  </StrictMode>\n);\n\n```\n\nsrc/vite-env.d.ts:\n```\n/// <reference types=\"vite/client\" />\n\n```\n\ntailwind.config.js:\n```\n/** @type {import('tailwindcss').Config} */\nexport default {\n  content: ['./index.html', './src/**/*.{js,ts,jsx,tsx}'],\n  darkMode: 'class',\n  theme: {\n    extend: {},\n  },\n  plugins: [],\n};\n```\n\ntsconfig.app.json:\n```\n{\n  \"compilerOptions\": {\n    \"target\": \"ES2020\",\n    \"useDefineForClassFields\": true,\n    \"lib\": [\"ES2020\", \"DOM\", \"DOM.Iterable\"],\n    \"module\": \"ESNext\",\n    \"skipLibCheck\": true,\n\n    /* Bundler mode */\n    \"moduleResolution\": \"bundler\",\n    \"allowImportingTsExtensions\": true,\n    \"isolatedModules\": true,\n    \"moduleDetection\": \"force\",\n    \"noEmit\": true,\n    \"jsx\": \"react-jsx\",\n\n    /* Linting */\n    \"strict\": true,\n    \"noUnusedLocals\": true,\n    \"noUnusedParameters\": true,\n    \"noFallthroughCasesInSwitch\": true\n  },\n  \"include\": [\"src\"]\n}\n\n```\n\ntsconfig.json:\n```\n{\n  \"files\": [],\n  \"references\": [\n    { \"path\": \"./tsconfig.app.json\" },\n    { \"path\": \"./tsconfig.node.json\" }\n  ]\n}\n\n```\n\ntsconfig.node.json:\n```\n{\n  \"compilerOptions\": {\n    \"target\": \"ES2022\",\n    \"lib\": [\"ES2023\"],\n    \"module\": \"ESNext\",\n    \"skipLibCheck\": true,\n\n    /* Bundler mode */\n    \"moduleResolution\": \"bundler\",\n    \"allowImportingTsExtensions\": true,\n    \"isolatedModules\": true,\n    \"moduleDetection\": \"force\",\n    \"noEmit\": true,\n\n    /* Linting */\n    \"strict\": true,\n    \"noUnusedLocals\": true,\n    \"noUnusedParameters\": true,\n    \"noFallthroughCasesInSwitch\": true\n  },\n  \"include\": [\"vite.config.ts\"]\n}\n\n```\n\nvite.config.ts:\n```\nimport { defineConfig } from 'vite';\nimport react from '@vitejs/plugin-react';\n\n// https://vitejs.dev/config/\nexport default defineConfig({\n  plugins: [react()],\n  optimizeDeps: {\n    exclude: ['lucide-react'],\n  },\n});\n\n```\n\nHere is a list of files that exist on the file system but are not being shown to you:\n\n  - .gitignore\n  - .bolt/prompt\n  - .bolt/config.json\n  - package-lock.json
"""

In [5]:
historyPrompt = """
Below is the conversation history, including all previous messages along with the most recent assistant response. Please reference this context to inform your future responses and maintain conversation continuity.
"""

In [6]:
list_history_prompts = []
last_llm_response = ""

In [7]:
latestQuery=""

In [8]:

# for backward compatibility, you can still use `https://api.deepseek.com/v1` as `base_url`.
response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": systemPrompt},
        {"role": "user", "content": codeBasePrompt},
        *([{"role": "user", "content": historyPrompt}, {"role": "assistant", "content": last_llm_response}] if historyPrompt else [])
        {"role": "user", "content": latestQuery},
  ],
    max_tokens=1024,
    temperature=0.0,
    stream=False
)

print(response.choices[0].message.content)

SyntaxError: invalid syntax (<ipython-input-8-dca3e1a29d5b>, line 8)

In [ ]:
errorDebugPrompt = """
The following are the errors encountered while running the code. Please analyze these errors thoroughly, identify the root cause based on the context provided use precise debugging techniques and best practices to modify the codebase accordingly. Implement additional improvements or optimizations to enhance the code's functionality and maintainability,"""

In [ ]:
errorMessage = """
Access to fetch at 'https://api.example.com/data' from origin 'http://localhost:3000' has been blocked by CORS policy: No 'Access-Control-Allow-Origin' header is present on the requested resource. If an opaque response serves your needs, set the request's mode to 'no-cors' to fetch the resource with CORS disabled.
"""

In [ ]:
# for backward compatibility, you can still use `https://api.deepseek.com/v1` as `base_url`.
response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": systemPrompt},
        {"role": "user", "content": codeBasePrompt},
        {"role": "user", "content": errorDebugPrompt+errorMessage+codeBasePrompt},
  ],
    max_tokens=8000,
    temperature=0.0,
    stream=False
)

print(response.choices[0].message.content)

In [ ]:
systemPrompt2 = """
You are ZenCode, an expert AI assistant specializing in generating React applications that strictly adhere to enterprise design standards and component libraries.

<repository_context>
You will receive a serialized context of the current repository's generated code state. This includes:
- File structure and contents of previously generated code
- Current state of all files and directories
- Previous modifications and their history
CRITICAL:
- All file operations must be performed relative to this existing context
- Maintain consistency with previously generated code
- Consider dependencies and relationships between existing files
- Ensure backwards compatibility when modifying existing files
- Debug issues encountered in the current code context.
- Provide a detailed explanation and resolution steps for any encountered errors.
</repository_context>

<system_constraints>
You are operating in WebContainer, an in-browser Node.js runtime that emulates a Linux system. Key limitations:
- Runs in browser, not a full Linux system
- Can only execute browser-compatible code (JS, WebAssembly)
- Python limited to standard library only (NO pip)
- No native binary execution or C/C++ compilation
- Git not available
- For databases, use browser-compatible options (libsql, sqlite)
- Handle and debug system-level error messages by providing actionable insights.
</system_constraints>

<enterprise_context>
You will receive:
1. Top-K relevant internal components from our vector database (Pinecone)
2. Global CSS standards and design system guidelines
3. Approved list of npm packages
CRITICAL REQUIREMENTS:
- ONLY use provided internal components from the enterprise library
- Strictly follow enterprise design standards
- Only use approved npm packages. Do not use any external npm packages not used in the given context.
- Focus on rapid prototyping by reusing existing components
- Maintain consistent styling and UX patterns
- Validate and debug dependency issues within the approved npm packages
- Suggest improvements or refactorings when system or code issues arise.
</enterprise_context>

<response_format>
Generate responses in JSON format following the StepType enum:
- CreateFile (0): New files
- CreateFolder (1): New directories
- EditFile (2): Modify files
- DeleteFile (3): Remove files
Each step must include:
- id: Unique integer
- title: Step description
- description: Detailed explanation
- type: StepType enum value
- content: File content or command
- path: Target file/folder path
- Include debug steps as part of the description field for error-handling scenarios.
</response_format>

<code_formatting>
- Use 2 spaces for indentation
- Follow enterprise code style guide
- Split functionality into modular components
- Keep files small and focused
- Use proper TypeScript types
- Validate any error-prone functions or components using unit tests and ensure output consistency.
</code_formatting>

IMPORTANT:
1. Think holistically before generating responses
2. Consider all file dependencies and impacts
3. Always use latest file modifications
4. Install dependencies first
5. Never re-run dev server if already running
6. Provide complete, untruncated code
7. Focus on reusing enterprise components
8. Debug and resolve any error responses before finalizing outputs
9. Document the debugging process within the JSON response where applicable.

"""

In [ ]:
response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": systemPrompt2},
        {"role": "user", "content": codeBasePrompt},
        {"role": "user", "content": errorDebugPrompt+errorMessage+codeBasePrompt},
  ],
    max_tokens=8000,
    temperature=0.0,
    stream=False
)
print("response with updated sys prompt")
print(response.choices[0].message.content)

In [ ]:
model = OpenAIServerModel(
    model_id="deepseek-ai/DeepSeek-R1",
    api_base="https://api.together.xyz/v1/", # Leave this blank to query OpenAI servers.
    api_key=os.environ["TOGETHER_API_KEY"], # Switch to the API key for the server you're targeting.
)